# LNDL Types - Core Type Definitions

LNDL (Lion Directive Language) types provide structured representations for parsing and processing LNDL responses. This module defines the type system for variable declarations, action calls, and output validation.

**Core Types:**
- **LvarMetadata**: Variable declarations from `<lvar>` tags
- **LactMetadata**: Action declarations from `<lact>` tags
- **ParsedConstructor**: Type constructors from `OUT{}` blocks
- **ActionCall**: Parsed function/tool invocations
- **LNDLOutput**: Complete validated output with execution lifecycle

**Key Concepts:**
- **Partial Validation**: Models with ActionCall fields bypass validation until execution
- **Action Lifecycle**: Parse → Execute → Re-validate
- **Execution Guards**: Prevent persistence of unexecuted actions

In [1]:
from typing import Any

from pydantic import BaseModel, Field

from lionherd_core.lndl.types import (
    ActionCall,
    LactMetadata,
    LNDLOutput,
    LvarMetadata,
    ParsedConstructor,
    ensure_no_action_calls,
    has_action_calls,
    revalidate_with_action_results,
)

## 1. LvarMetadata - Variable Declarations

Represents parsed variable declarations from `<lvar>` tags. Variables can be namespaced to model fields or standalone.

**Format:** `<lvar [Model.field] local_name>value</lvar>`

**Use Cases:**
- Namespaced: `<lvar Report.title title>Analysis Report</lvar>`
- Direct: `<lvar user_id>12345</lvar>`

In [2]:
# Namespaced variable - bound to model field
namespaced = LvarMetadata(
    model="Report", field="title", local_name="title", value="Analysis Report"
)

print(f"Model: {namespaced.model}")
print(f"Field: {namespaced.field}")
print(f"Local name: {namespaced.local_name}")
print(f"Value: {namespaced.value}")
print(f"Frozen: {namespaced.__dataclass_params__.frozen}")

Model: Report
Field: title
Local name: title
Value: Analysis Report
Frozen: True


In [3]:
# Immutable - cannot modify after creation
try:
    namespaced.value = "New Value"
except Exception as e:
    print(f"✓ Immutable: {type(e).__name__}: {e}")

✓ Immutable: FrozenInstanceError: cannot assign to field 'value'


## 2. LactMetadata - Action Declarations

Represents parsed action declarations from `<lact>` tags. Actions can be namespaced to model fields or direct function calls.

**Format:** `<lact [Model.field] local_name>function_call</lact>`

**Use Cases:**
- Namespaced: `<lact Report.summary s>generate_summary(text=article)</lact>`
- Direct: `<lact search>web_search(query="AI trends")</lact>`

In [4]:
# Namespaced action - bound to model field
namespaced_action = LactMetadata(
    model="Report",
    field="summary",
    local_name="s",
    call="generate_summary(text=article, max_words=150)",
)

print(f"Model: {namespaced_action.model}")
print(f"Field: {namespaced_action.field}")
print(f"Local name: {namespaced_action.local_name}")
print(f"Call: {namespaced_action.call}")

Model: Report
Field: summary
Local name: s
Call: generate_summary(text=article, max_words=150)


In [5]:
# Direct action - standalone function call
direct_action = LactMetadata(
    model=None, field=None, local_name="search", call='web_search(query="latest AI research")'
)

print(f"Model: {direct_action.model}")
print(f"Field: {direct_action.field}")
print(f"Local name: {direct_action.local_name}")
print(f"Is namespaced: {direct_action.model is not None}")

Model: None
Field: None
Local name: search
Is namespaced: False


## 3. ParsedConstructor - Type Constructors

Represents parsed type constructors from `OUT{}` blocks. Captures class name, keyword arguments, and raw syntax.

**Format:** `OUT{ClassName(field1=value1, field2=value2)}`

**Features:**
- Detects `**dict` unpacking syntax
- Preserves raw constructor string
- Supports nested constructors

In [6]:
# Basic constructor
constructor = ParsedConstructor(
    class_name="Report",
    kwargs={"title": "Analysis", "author": "Alice", "status": "draft"},
    raw="Report(title='Analysis', author='Alice', status='draft')",
)

print(f"Class: {constructor.class_name}")
print(f"Args: {constructor.kwargs}")
print(f"Raw: {constructor.raw}")
print(f"Has dict unpack: {constructor.has_dict_unpack}")

Class: Report
Args: {'title': 'Analysis', 'author': 'Alice', 'status': 'draft'}
Raw: Report(title='Analysis', author='Alice', status='draft')
Has dict unpack: False


In [7]:
# Constructor with **dict unpacking
unpacking_constructor = ParsedConstructor(
    class_name="Report",
    kwargs={"**metadata": None},  # Placeholder for unpacking
    raw="Report(**metadata)",
)

print(f"Class: {unpacking_constructor.class_name}")
print(f"Has dict unpack: {unpacking_constructor.has_dict_unpack}")
print(f"Unpack keys: {[k for k in unpacking_constructor.kwargs if k.startswith('**')]}")

Class: Report
Has dict unpack: True
Unpack keys: ['**metadata']


## 4. ActionCall - Parsed Function Invocations

Represents a tool/function call that needs execution. ActionCall objects serve as placeholders in models until actions are executed.

**Attributes:**
- `name`: Local reference (from `<lact>` local_name)
- `function`: Function/tool to invoke
- `arguments`: Parsed kwargs dict
- `raw_call`: Original Python call string

**Lifecycle:** Only actions referenced in `OUT{}` blocks are executed.

In [8]:
# Create an action call
action = ActionCall(
    name="summarize",
    function="generate_summary",
    arguments={"text": "Long article...", "max_words": 150},
    raw_call="generate_summary(text='Long article...', max_words=150)",
)

print(f"Name: {action.name}")
print(f"Function: {action.function}")
print(f"Arguments: {action.arguments}")
print(f"Raw call: {action.raw_call}")

Name: summarize
Function: generate_summary
Arguments: {'text': 'Long article...', 'max_words': 150}
Raw call: generate_summary(text='Long article...', max_words=150)


In [9]:
# ActionCall as placeholder in model
class Report(BaseModel):
    title: str
    summary: str  # Will temporarily hold ActionCall


# Partial construction with ActionCall (bypasses validation)
report = Report.model_construct(
    title="AI Research Report",
    summary=action,  # ActionCall placeholder
)

print(f"Title: {report.title}")
print(f"Summary type: {type(report.summary).__name__}")
print(f"Summary is ActionCall: {isinstance(report.summary, ActionCall)}")

Title: AI Research Report
Summary type: ActionCall
Summary is ActionCall: True


## 5. has_action_calls - Detection Utility

Recursively checks if a BaseModel instance contains any ActionCall objects, including nested models and collections.

**Use Cases:**
- Pre-persistence validation
- Conditional re-validation logic
- Debugging action execution state

In [10]:
# Model without actions
clean_report = Report(title="Report", summary="Completed summary")
print(f"Clean report has actions: {has_action_calls(clean_report)}")

# Model with ActionCall placeholder
pending_report = Report.model_construct(
    title="Report",
    summary=ActionCall(name="s", function="summarize", arguments={}, raw_call="summarize()"),
)
print(f"Pending report has actions: {has_action_calls(pending_report)}")

Clean report has actions: False
Pending report has actions: True


In [11]:
# Nested models detection
class Section(BaseModel):
    title: str
    content: str


class NestedReport(BaseModel):
    main_report: Report
    sections: list[Section]


nested = NestedReport.model_construct(
    main_report=pending_report,  # Contains ActionCall
    sections=[Section(title="Intro", content="...")],
)

print(f"Nested report has actions: {has_action_calls(nested)}")
print("Detects actions at any depth: ✓")

Nested report has actions: True
Detects actions at any depth: ✓


## 6. ensure_no_action_calls - Persistence Guard

Validation guard that prevents persistence of models with unexecuted ActionCall objects. Raises ValueError with field paths if actions detected.

**Critical:** Always use before database writes, API responses, or cache storage.

**Error Details:**
- Lists field paths containing ActionCall objects
- Shows up to 3 fields (+ count if more)
- Provides clear remediation message

In [12]:
# Valid model passes guard
validated_report = Report(title="Report", summary="Final summary")
safe_report = ensure_no_action_calls(validated_report)
print(f"✓ Guard passed: {safe_report.title}")

✓ Guard passed: Report


In [13]:
# Model with ActionCall fails guard
unsafe_report = Report.model_construct(
    title="Report",
    summary=ActionCall(name="s", function="summarize", arguments={}, raw_call="summarize()"),
)

try:
    ensure_no_action_calls(unsafe_report)
except ValueError as e:
    print("✓ Guard prevented persistence")
    print(f"Error message:\n{e}")

✓ Guard prevented persistence
Error message:
Report contains unexecuted actions in fields: summary. Models with ActionCall placeholders must be re-validated after action execution. Call revalidate_with_action_results() before using this model.


In [14]:
# Multiple ActionCall fields in nested model
class ComplexReport(BaseModel):
    title: str
    summary: str
    conclusion: str
    sections: list[str]


action1 = ActionCall(name="s1", function="summarize", arguments={}, raw_call="summarize()")
action2 = ActionCall(name="s2", function="conclude", arguments={}, raw_call="conclude()")
action3 = ActionCall(name="s3", function="analyze", arguments={}, raw_call="analyze()")

complex_report = ComplexReport.model_construct(
    title="Report",
    summary=action1,
    conclusion=action2,
    sections=[action3],  # ActionCall in list
)

try:
    ensure_no_action_calls(complex_report)
except ValueError as e:
    error_msg = str(e)
    print(f"Detected multiple fields: {error_msg.split(':')[1].split('.')[0]}")
    print(f"Full error:\n{e}")

Detected multiple fields:  summary, conclusion, sections[0]
Full error:
ComplexReport contains unexecuted actions in fields: summary, conclusion, sections[0]. Models with ActionCall placeholders must be re-validated after action execution. Call revalidate_with_action_results() before using this model.


## 7. revalidate_with_action_results - Action Integration

Replaces ActionCall placeholders with execution results and performs full Pydantic validation.

**Process:**
1. Extract current field values
2. Replace ActionCall objects with results from action_results dict
3. Reconstruct model with full validation

**Raises:**
- ValueError: Action result not found in results dict
- ValidationError: Result doesn't satisfy field constraints

In [15]:
# Create model with ActionCall placeholder
pending = Report.model_construct(
    title="AI Analysis",
    summary=ActionCall(
        name="summarize",
        function="generate_summary",
        arguments={"text": "..."},
        raw_call="generate_summary(text='...')",
    ),
)

print(f"Before: summary type = {type(pending.summary).__name__}")

# Execute action (simulated)
action_results = {"summarize": "This is a comprehensive analysis of AI trends in 2025."}

# Re-validate with results
validated = revalidate_with_action_results(pending, action_results)

print(f"After: summary type = {type(validated.summary).__name__}")
print(f"After: summary value = {validated.summary}")

Before: summary type = ActionCall
After: summary type = str
After: summary value = This is a comprehensive analysis of AI trends in 2025.


/Users/lion/projects/open-source/lionherd-core/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='summary', input_value=ActionCall(name='summariz...te_summary(text='...')"), input_type=ActionCall])
  return self.__pydantic_serializer__.to_python(


In [16]:
# Missing action result raises error
incomplete_results = {}  # Empty dict - no results

try:
    revalidate_with_action_results(pending, incomplete_results)
except ValueError as e:
    print("✓ Missing result detected")
    print(f"Error: {e}")

✓ Missing result detected
Error: Action 'summarize' in field 'summary' has no execution result. Available results: []


In [17]:
# Validation errors on result substitution
class ValidatedReport(BaseModel):
    title: str = Field(min_length=1)
    summary: str = Field(min_length=10)  # Minimum 10 characters


pending_validated = ValidatedReport.model_construct(
    title="Report",
    summary=ActionCall(name="s", function="summarize", arguments={}, raw_call="summarize()"),
)

# Result too short - violates constraint
invalid_results = {"s": "Short"}  # Only 5 characters

try:
    revalidate_with_action_results(pending_validated, invalid_results)
except Exception as e:
    print(f"✓ Validation enforced: {type(e).__name__}")
    print("Constraint violation detected")

✓ Validation enforced: ValidationError
Constraint violation detected


/Users/lion/projects/open-source/lionherd-core/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='summary', input_value=ActionCall(name='s', func... raw_call='summarize()'), input_type=ActionCall])
  return self.__pydantic_serializer__.to_python(


## 8. LNDLOutput - Complete Output Structure

Container for validated LNDL parse results with action execution lifecycle.

**Fields:**
- `fields`: BaseModel instances or ActionCall objects (pre-execution)
- `lvars`: Variable declarations (debugging)
- `lacts`: All action declarations (debugging)
- `actions`: Actions referenced in OUT{} (pending execution)
- `raw_out_block`: Original OUT{} string (debugging)

**Access:**
- Dictionary: `output['field_name']`
- Attribute: `output.field_name`

In [18]:
# Create LNDLOutput
output = LNDLOutput(
    fields={
        "report": Report.model_construct(
            title="Analysis",
            summary=ActionCall(
                name="summarize",
                function="generate_summary",
                arguments={"max_words": 150},
                raw_call="generate_summary(max_words=150)",
            ),
        )
    },
    lvars={
        "title": LvarMetadata(model="Report", field="title", local_name="title", value="Analysis")
    },
    lacts={
        "summarize": LactMetadata(
            model="Report",
            field="summary",
            local_name="summarize",
            call="generate_summary(max_words=150)",
        )
    },
    actions={
        "summarize": ActionCall(
            name="summarize",
            function="generate_summary",
            arguments={"max_words": 150},
            raw_call="generate_summary(max_words=150)",
        )
    },
    raw_out_block="OUT{Report(title=title, summary=summarize)}",
)

print(f"Fields: {list(output.fields.keys())}")
print(f"Actions to execute: {list(output.actions.keys())}")
print(f"Raw OUT block: {output.raw_out_block}")

Fields: ['report']
Actions to execute: ['summarize']
Raw OUT block: OUT{Report(title=title, summary=summarize)}


In [19]:
# Dictionary-style access
report_dict = output["report"]
print(f"Dict access type: {type(report_dict).__name__}")
print(f"Dict access title: {report_dict.title}")

# Attribute-style access
report_attr = output.report
print(f"Attr access type: {type(report_attr).__name__}")
print(f"Same object: {report_dict is report_attr}")

Dict access type: Report
Dict access title: Analysis
Attr access type: Report
Same object: True


## 9. Complete Action Execution Lifecycle

Demonstrates the full workflow from parsing to validated output.

**Steps:**
1. Parse LNDL response → LNDLOutput with ActionCall placeholders
2. Execute actions using .actions dict
3. Re-validate models with action results
4. Guard before persistence

In [20]:
# Step 1: Parse (simulated - would come from parser)
parsed_output = LNDLOutput(
    fields={
        "analysis": Report.model_construct(
            title="Market Analysis",
            summary=ActionCall(
                name="summarize",
                function="generate_summary",
                arguments={"source": "market_data"},
                raw_call="generate_summary(source='market_data')",
            ),
        )
    },
    lvars={},
    lacts={},
    actions={
        "summarize": ActionCall(
            name="summarize",
            function="generate_summary",
            arguments={"source": "market_data"},
            raw_call="generate_summary(source='market_data')",
        )
    },
    raw_out_block="OUT{Report(title='Market Analysis', summary=summarize)}",
)

print("Step 1: Parsed LNDL output")
print(f"  Has actions: {has_action_calls(parsed_output.analysis)}")
print(f"  Actions to execute: {list(parsed_output.actions.keys())}")

Step 1: Parsed LNDL output
  Has actions: True
  Actions to execute: ['summarize']


In [21]:
# Step 2: Execute actions
def execute_tool(function: str, arguments: dict[str, Any]) -> str:
    """Simulated tool execution."""
    if function == "generate_summary":
        return f"Summary generated from {arguments.get('source', 'unknown source')}"
    return "Unknown function"


action_results = {}
for name, action in parsed_output.actions.items():
    result = execute_tool(action.function, action.arguments)
    action_results[name] = result
    print(f"Executed {name}: {result}")

print(f"\nStep 2: Collected {len(action_results)} results")

Executed summarize: Summary generated from market_data

Step 2: Collected 1 results


In [22]:
# Step 3: Re-validate with results
validated_analysis = revalidate_with_action_results(parsed_output.analysis, action_results)

print("Step 3: Re-validated model")
print(f"  Summary type: {type(validated_analysis.summary).__name__}")
print(f"  Summary value: {validated_analysis.summary}")
print(f"  Has actions: {has_action_calls(validated_analysis)}")

Step 3: Re-validated model
  Summary type: str
  Summary value: Summary generated from market_data
  Has actions: False


/Users/lion/projects/open-source/lionherd-core/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='summary', input_value=ActionCall(name='summariz...(source='market_data')"), input_type=ActionCall])
  return self.__pydantic_serializer__.to_python(


In [23]:
# Step 4: Guard before persistence
safe_analysis = ensure_no_action_calls(validated_analysis)
print("Step 4: Persistence guard passed")
print("  Safe to persist: ✓")
print(f"  Title: {safe_analysis.title}")
print(f"  Summary: {safe_analysis.summary}")

# Attempting to persist original would fail
try:
    ensure_no_action_calls(parsed_output.analysis)
except ValueError:
    print("  Original would fail guard: ✓")

Step 4: Persistence guard passed
  Safe to persist: ✓
  Title: Market Analysis
  Summary: Summary generated from market_data
  Original would fail guard: ✓


## Summary Checklist

**LNDL Type System:**
- ✅ **LvarMetadata**: Immutable variable declarations from `<lvar>` tags
- ✅ **LactMetadata**: Immutable action declarations from `<lact>` tags
- ✅ **ParsedConstructor**: Type constructors with `**dict` detection
- ✅ **ActionCall**: Executable function placeholders
- ✅ **LNDLOutput**: Complete parse result with lifecycle management

**Validation Lifecycle:**
- ✅ **Partial validation** during parse (ActionCall bypasses constraints)
- ✅ **Action execution** using .actions dict
- ✅ **Re-validation** after substituting results
- ✅ **Persistence guard** prevents unexecuted action corruption

**Utilities:**
- ✅ `has_action_calls()`: Recursive detection (nested models + collections)
- ✅ `ensure_no_action_calls()`: Pre-persistence validation guard
- ✅ `revalidate_with_action_results()`: Full validation after execution

**Key Patterns:**
- ✅ Always execute actions before validation
- ✅ Always guard before persistence
- ✅ Use `model_construct()` for partial validation
- ✅ Use `revalidate_with_action_results()` for full validation

**Next Steps:**
- See `lndl_parser.ipynb` for LNDL response parsing
- See `lndl_resolver.ipynb` for action execution integration
- See `lndl_fuzzy.ipynb` for fuzzy validation strategies